# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from key import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
csv_path = "Output/city_weather_data.csv"
city_weather_df  = pd.read_csv(csv_path, encoding="utf-8")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sitka,57.05,-135.33,57.20,50,90,11.41,US,1591757113
1,punta arenas,-53.15,-70.92,33.80,86,50,5.82,CL,1591757113
2,lompoc,34.64,-120.46,71.60,43,0,11.41,US,1591756927
3,ust-ishim,57.69,71.17,76.35,52,45,5.57,RU,1591757113
4,bluff,-46.60,168.33,54.00,79,39,11.01,NZ,1591757113
...,...,...,...,...,...,...,...,...,...
544,baturaja,-4.13,104.17,84.74,68,65,4.47,ID,1591757184
545,kochevo,59.60,54.31,69.48,58,0,2.91,RU,1591757184
546,clearwater,27.97,-82.80,82.99,78,1,4.47,US,1591757184
547,port hawkesbury,45.62,-61.35,44.01,100,20,4.70,CA,1591757184


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=gkey)

In [18]:
locations = city_weather_df[["Lat", "Lng"]].astype(float)
humidity = city_weather_df["Humidity"]

In [19]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Narrow down by ideal temp
ideal_temp_df = city_weather_df.loc[(city_weather_df['Max Temp'] > 70) & (city_weather_df['Max Temp'] < 80)]
ideal_temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,lompoc,34.64,-120.46,71.60,43,0,11.41,US,1591756927
3,ust-ishim,57.69,71.17,76.35,52,45,5.57,RU,1591757113
5,arraial do cabo,-22.97,-42.03,72.39,88,1,19.17,BR,1591757114
11,quatre cocos,-20.21,57.76,71.60,73,75,12.75,MU,1591757114
13,rikitea,-23.12,-134.97,75.81,84,1,12.03,PF,1591757115
...,...,...,...,...,...,...,...,...,...
525,kyabe,9.45,18.94,76.41,73,100,12.93,TD,1591757182
533,shimanovsk,52.00,127.70,73.24,31,0,9.60,RU,1591757183
540,rockhampton,-23.38,150.50,73.40,60,75,10.29,AU,1591757183
541,yeppoon,-23.13,150.73,73.40,60,75,10.29,AU,1591757184


In [21]:
# Narrow down by ideal wind speed
ideal_wind_df = ideal_temp_df.loc[ideal_temp_df['Wind Speed'] < 10]
ideal_wind_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,ust-ishim,57.69,71.17,76.35,52,45,5.57,RU,1591757113
17,vaini,-21.20,-175.20,78.80,83,75,9.17,TO,1591757115
32,pemba,-12.97,40.52,73.40,88,75,8.05,MZ,1591757117
47,jamestown,42.10,-79.24,77.00,53,1,4.70,US,1591757119
49,itarema,-2.92,-39.92,76.80,85,0,7.83,BR,1591757119
...,...,...,...,...,...,...,...,...,...
515,tuxpan,19.55,-103.40,70.20,85,94,5.30,MX,1591757181
523,luorong,24.41,109.61,79.57,90,97,5.77,CN,1591757181
524,port said,31.26,32.28,72.63,72,0,5.37,EG,1591757164
533,shimanovsk,52.00,127.70,73.24,31,0,9.60,RU,1591757183


In [22]:
# Narrow down by ideal humidity
ideal_hum_df = ideal_wind_df.loc[ideal_wind_df['Humidity'] < 50]
ideal_hum_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,carnarvon,-24.87,113.63,74.12,27,0,1.16,AU,1591757121
73,durango,24.83,-104.83,78.84,23,78,3.47,MX,1591756984
137,zalantun,48.00,122.72,72.28,36,100,7.20,CN,1591757131
141,dharchula,29.85,80.53,78.30,46,27,3.15,IN,1591757132
181,baghdad,33.34,44.40,78.80,34,0,1.12,IQ,1591756981
190,kondinskoye,59.65,67.41,76.80,49,55,7.76,RU,1591757138
222,winnemucca,40.97,-117.74,75.20,17,1,9.17,US,1591756922
224,srednekolymsk,67.45,153.68,75.20,32,100,0.51,RU,1591757143
292,san quintin,30.48,-115.95,70.79,38,0,9.66,MX,1591757152
308,parkent,41.29,69.68,71.60,43,51,9.17,UZ,1591757154


In [23]:
# Narrow down by ideal cloudiness level
hotel_df = ideal_hum_df.loc[ideal_hum_df['Cloudiness'] < 50]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,carnarvon,-24.87,113.63,74.12,27,0,1.16,AU,1591757121
141,dharchula,29.85,80.53,78.30,46,27,3.15,IN,1591757132
181,baghdad,33.34,44.40,78.80,34,0,1.12,IQ,1591756981
222,winnemucca,40.97,-117.74,75.20,17,1,9.17,US,1591756922
292,san quintin,30.48,-115.95,70.79,38,0,9.66,MX,1591757152
311,yenotayevka,47.25,47.03,76.64,36,0,6.82,RU,1591757154
335,lubbock,33.60,-101.83,71.60,26,1,9.17,US,1591756897
351,shache,38.42,77.24,79.12,20,0,5.50,CN,1591757159
365,semnan,35.57,53.40,77.18,22,0,5.88,IR,1591756903
436,zhezkazgan,47.80,67.71,73.40,31,6,8.95,KZ,1591757170


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# add Hotel Name column
hotel_df['Hotel Name'] = ''
hotel_df

<ipython-input-24-141ef324a071>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,carnarvon,-24.87,113.63,74.12,27,0,1.16,AU,1591757121,
141,dharchula,29.85,80.53,78.30,46,27,3.15,IN,1591757132,
181,baghdad,33.34,44.40,78.80,34,0,1.12,IQ,1591756981,
222,winnemucca,40.97,-117.74,75.20,17,1,9.17,US,1591756922,
292,san quintin,30.48,-115.95,70.79,38,0,9.66,MX,1591757152,
311,yenotayevka,47.25,47.03,76.64,36,0,6.82,RU,1591757154,
335,lubbock,33.60,-101.83,71.60,26,1,9.17,US,1591756897,
351,shache,38.42,77.24,79.12,20,0,5.50,CN,1591757159,
365,semnan,35.57,53.40,77.18,22,0,5.88,IR,1591756903,
436,zhezkazgan,47.80,67.71,73.40,31,6,8.95,KZ,1591757170,


In [28]:
# find the closest hotel for each city

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": "5000",
    "type": "hotel",
    "key": gkey,
}

# iterate through the dataframe
for index, row in hotel_df.iterrows():

    # get location data from DF
    location = (f"{row['Lat']}, {row['Lng']}")

    # add locatoin to params dictionary
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for: {row['City'].title()}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[1]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# results[0].keys() - dict_keys(['geometry', 'icon', 'id', 'name', 'photos', 'place_id', 'reference', 'scope', 'types', 'vicinity'])   
# print(json.dumps(results, indent=4))  

Retrieving Results for: Carnarvon.
Closest hotel is Hospitality Carnarvon.
------------
Retrieving Results for: Dharchula.
Closest hotel is District Police Office.
------------
Retrieving Results for: Baghdad.
Closest hotel is Republican Palace.
------------
Retrieving Results for: Winnemucca.
Closest hotel is Winners Inn Casino.
------------
Retrieving Results for: San Quintin.
Closest hotel is Pemex.
------------
Retrieving Results for: Yenotayevka.
Closest hotel is Rosgosstrakh.
------------
Retrieving Results for: Lubbock.
Closest hotel is La Quinta Inn by Wyndham Lubbock - Downtown Civic Center.
------------
Retrieving Results for: Shache.
Closest hotel is Shucaicun.
------------
Retrieving Results for: Semnan.
Closest hotel is Mellat Bank.
------------
Retrieving Results for: Zhezkazgan.
Closest hotel is Hotel Zumrat.
------------
Retrieving Results for: Mulege.
Closest hotel is Casa Granada.
------------
Retrieving Results for: Chardara.
Closest hotel is Apparat Akima Shardarins

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Add info box layer on heat map

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))